In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [38]:
rating_data = pd.read_csv("D:\dataset\Rating.csv")
movies_data = pd.read_csv("D:\dataset\movies.csv")

In [39]:
print(rating_data.head())
print(movies_data.head())

   userId  movieId  rating   timestamp
0       1      296     5.0  1147880044
1       1      306     3.5  1147868817
2       1      307     5.0  1147868828
3       1      665     5.0  1147878820
4       1      899     3.5  1147868510
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


Merge rating and movies data and drop unnecessary columns

In [40]:
merge_data = pd.merge(rating_data,movies_data)
merge_data= merge_data.drop(['timestamp','genres'], axis=1)

In [41]:
merge_data

,userId,movieId,rating,title
0,1,296,5.0,Pulp Fiction (1994)
1,3,296,5.0,Pulp Fiction (1994)
2,4,296,4.0,Pulp Fiction (1994)
3,5,296,4.0,Pulp Fiction (1994)
4,7,296,4.0,Pulp Fiction (1994)
...,...,...,...,...
13291,100,714,5.0,Dead Man (1995)
13292,100,743,3.0,Spy Hard (1996)
13293,100,1365,3.0,Ridicule (1996)
13294,100,1366,4.0,"Crucible, The (1996)"


Create user time matrix

In [62]:
matrix = merge_data.pivot_table(index='userId', columns='title', values='rating')

In [63]:
matrix

title,$5 a Day (2008),'Round Midnight (1986),"'burbs, The (1989)",(500) Days of Summer (2009),(Dis)Honesty: The Truth About Lies (2015),...And Justice for All (1979),10 (1979),10 Rillington Place (1971),10 Things I Hate About You (1999),"10,000 BC (2008)",...,Zack and Miri Make a Porno (2008),Zelig (1983),Zero Effect (1998),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,3.5,NaN,NaN,3.5,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,4.5,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
item_similarity = cosine_similarity(matrix.fillna(0))
np.fill_diagonal(item_similarity, 0)

In [1]:
def get_recommendations(user, top=5):
    # Get the user's ratings and item similarity
    user_ratings = matrix.loc[user].values.reshape(-1, 1)
    similarity_scores = item_similarity.dot(user_ratings.T).reshape(-1)

    # Sort the movies based on similarity scores
    sorted_indices = np.argsort(similarity_scores)[::-1]
    top_indices = sorted_indices[:top]

    # Get the recommended movie IDs
    recommended_movie_ids = user_item_matrix.columns[top_indices]

    # Get the movie titles
    recommended_movies = movies_data[movies_data['movieId'].isin(recommended_movie_ids)]

    return recommended_movies